In [1]:
import pandas as  pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MaxAbsScaler, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC
from xgboost import XGBRegressor 
from imblearn.combine import SMOTETomek
from sklearn.metrics import *

In [2]:
df= pd.read_csv('Dataset.csv')
df.head()

,Unnamed: 0,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
0,0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...
1,1,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...
2,2,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,31/08/22,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
3,3,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0,NaN,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...
4,4,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Junk lead,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,fc10fffd29cfbe93c55158fb47752a7501c211d253468c...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46608 entries, 0 to 46607
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      46608 non-null  int64 
 1   Agent_id        46608 non-null  object
 2   status          46608 non-null  object
 3   lost_reason     43244 non-null  object
 4   budget          42908 non-null  object
 5   lease           44267 non-null  object
 6   movein          32970 non-null  object
 7   source          46608 non-null  object
 8   source_city     46608 non-null  object
 9   source_country  46608 non-null  object
 10  utm_source      46608 non-null  object
 11  utm_medium      46608 non-null  object
 12  des_city        46608 non-null  object
 13  des_country     46608 non-null  object
 14  room_type       23061 non-null  object
 15  lead_id         46608 non-null  object
dtypes: int64(1), object(15)
memory usage: 5.7+ MB


In [4]:
df= df.drop(['Unnamed: 0','Agent_id','source', 'source_city', 'source_country', 'utm_source',
       'utm_medium', 'des_city', 'des_country','lead_id'],axis=1)

In [5]:
df.head()

,status,lost_reason,budget,lease,movein,room_type
0,LOST,Not responding,NaN,NaN,NaN,NaN
1,LOST,Low budget,NaN,NaN,NaN,NaN
2,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,31/08/22,Ensuite
3,LOST,Low budget,0-0,0,NaN,NaN
4,LOST,Junk lead,NaN,NaN,NaN,NaN


In [6]:
temp= pd.DataFrame()
for i in ['LOST', 'WON', 'OPPORTUNITY', 'CONTACTED', 'PROCESSING',
       'IMPORTANT']:
    if i == 'WON':
        temp= temp.append(df[df['status'] == i])
    elif i == 'LOST':
        temp= temp.append(df[df['status'] == i])   

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\4118114261.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp= temp.append(df[df['status'] == i])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\4118114261.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp= temp.append(df[df['status'] == i])


In [7]:
temp.status.value_counts()

LOST    43244
WON      3073
Name: status, dtype: int64

# Data Cleaning and Feature Engineering

In [8]:
temp.budget.unique()

array([nan, '£121 - £180 Per Week', '0-0', ..., '208',
       '0-200 to 300pounds', '£135 Per week'], dtype=object)

In [9]:
temp['budget']= temp.budget.str.replace(' ','').str.replace('Per',' ').str.split(' ',n=1,expand=True)[0].str.replace('per',' ').str.split(' ',n=1,expand=True)[0]

In [10]:
temp['budget']=temp['budget'].str.replace('£','').str.replace('/',' ').str.split(' ',n=1,expand= True)[0]

In [11]:
temp['budget']=temp['budget'].str.split('-',n=1,expand=True)[0]


In [12]:
df= temp[temp['budget'].str.isnumeric()== True]
    

In [13]:
df.lease.unique()

array(['Full Year Course Stay 40 - 44 weeks', '0', 'semester-stay', '51',
       'Semester Stay 20 - 24 weeks',
       'Complete Education Year Stay 50 - 52 weeks', 'full-course-stay',
       'full-year-stay', 'Other', '51 weeks', '44', '18', '51weeks',
       'Summer/Short Stay 8 - 12 weeks', '51 weeks \\', 'other',
       'short-stay', '52 weeks', '44 weeks', '12', '96', '43', '20',
       'full year stay', '1 year', '294', '20 - 24 weeks', '1 year .',
       '48', '6', 'Academic year', '8', '51Weeks', '50 weeks', nan, '84',
       '15 weeks', '45', '39', '1', '50', '12-24', '25', '45 weeks',
       '44-51', '6 weeks', '24-36', '32', '44/week', '42', '40',
       '3 month', '44-51 weeks', '6 months', '2 years', '10', '52',
       '2 weeks', '46', '13', 'lease', 'short stay', '4', '52weeks',
       'FULL YEAR', '69', '16', '50 - 52 weeks', '44 - 51 weeks',
       '4-8 weeks', '4 weeks', 'full course', '3', '24', 'Full year stay',
       '11', '10-15 weeks', '40-44', '26', '40 - 44 wee

In [14]:
df['stay_type']=df.lease.str.replace('[^A-Za-z]','',regex= True).str.lower()

def FE(text):
    if isinstance(text, float): 
        return 0
    else:
        if 'year' in text:
            return 1
        elif 'semester' in text:
            return 2
        else:
            return 0
df['stay_type']=df['stay_type'].apply(FE)        

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\1046899661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stay_type']=df.lease.str.replace('[^A-Za-z]','',regex= True).str.lower()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\1046899661.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stay_type']=df['stay_type'].apply(FE)


In [15]:
df.head()

,status,lost_reason,budget,lease,movein,room_type,stay_type
2,LOST,Not responding,121,Full Year Course Stay 40 - 44 weeks,31/08/22,Ensuite,1
3,LOST,Low budget,0,0,NaN,NaN,0
5,LOST,Wants private accommodation,120,semester-stay,NaN,NaN,2
6,LOST,Short stay,0,0,NaN,NaN,0
7,LOST,Wants private accommodation,181,51,08/09/22,Entire Place,0


In [16]:
df.lease=df.lease.str.split('weeks',n=1, expand= True)[0].str.replace('Full Year Course Stay','').str.replace('Complete Education Year Stay','').str.replace('Semester Stay','').str.replace('Summer/Short Stay','')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\2420389272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.lease=df.lease.str.split('weeks',n=1, expand= True)[0].str.replace('Full Year Course Stay','').str.replace('Complete Education Year Stay','').str.replace('Semester Stay','').str.replace('Summer/Short Stay','')


In [17]:
df.lease= df.lease.str.split('-',n=1,expand= True)[0].str.replace(' ','')
df.lease.count()
df.lease=df.lease.str.replace('[^0-9]','',regex= True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\287772829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.lease= df.lease.str.split('-',n=1,expand= True)[0].str.replace(' ','')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\287772829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.lease=df.lease.str.replace('[^0-9]','',regex= True)


In [18]:
df.budget=df.budget.astype(int)
df.stay_type= df.stay_type.astype(int)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\88711404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.budget=df.budget.astype(int)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\88711404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.stay_type= df.stay_type.astype(int)


In [19]:
def remove(text):
    if isinstance(text, float): 
        return ''
    else:
        return text
df.lease= df.lease.apply(remove)    


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8596\2579332941.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.lease= df.lease.apply(remove)


In [20]:
df= df[df.lease != '']

In [21]:
df.lease= df.lease.astype(int)

In [22]:
df.dtypes

status         object
lost_reason    object
budget          int32
lease           int32
movein         object
room_type      object
stay_type       int32
dtype: object

In [23]:
df.head()

,status,lost_reason,budget,lease,movein,room_type,stay_type
2,LOST,Not responding,121,40,31/08/22,Ensuite,1
3,LOST,Low budget,0,0,NaN,NaN,0
6,LOST,Short stay,0,0,NaN,NaN,0
7,LOST,Wants private accommodation,181,51,08/09/22,Entire Place,0
8,LOST,Low budget,100,20,NaN,NaN,2


In [24]:
df=df.drop('room_type',axis=1)
df.head()

,status,lost_reason,budget,lease,movein,stay_type
2,LOST,Not responding,121,40,31/08/22,1
3,LOST,Low budget,0,0,NaN,0
6,LOST,Short stay,0,0,NaN,0
7,LOST,Wants private accommodation,181,51,08/09/22,0
8,LOST,Low budget,100,20,NaN,2


In [25]:
df.movein= df.movein.fillna(0)

df= df[df['movein']!= 0]

In [26]:
df.movein=pd.to_datetime(df.movein)
df['movein_day']= df.movein.dt.day
df['movein_month']= df.movein.dt.month
df['movein_year']= df.movein.dt.year


In [27]:
df.head()

,status,lost_reason,budget,lease,movein,stay_type,movein_day,movein_month,movein_year
2,LOST,Not responding,121,40,2022-08-31,1,31,8,2022
7,LOST,Wants private accommodation,181,51,2022-08-09,0,9,8,2022
11,LOST,Not responding,121,40,2022-09-15,1,15,9,2022
12,LOST,Wants private accommodation,121,40,2022-02-09,1,9,2,2022
13,LOST,Not responding,181,50,2022-04-09,1,9,4,2022


In [28]:
df= df.drop('movein',axis=1)
df.reset_index(drop= True, inplace= True)

df['movein_day']=df['movein_day'].astype(int)
df['movein_month']=df['movein_month'].astype(int)
df['movein_year']=df['movein_year'].astype(int)
df.head()

,status,lost_reason,budget,lease,stay_type,movein_day,movein_month,movein_year
0,LOST,Not responding,121,40,1,31,8,2022
1,LOST,Wants private accommodation,181,51,0,9,8,2022
2,LOST,Not responding,121,40,1,15,9,2022
3,LOST,Wants private accommodation,121,40,1,9,2,2022
4,LOST,Not responding,181,50,1,9,4,2022


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26909 entries, 0 to 26908
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status        26909 non-null  object
 1   lost_reason   24663 non-null  object
 2   budget        26909 non-null  int32 
 3   lease         26909 non-null  int32 
 4   stay_type     26909 non-null  int32 
 5   movein_day    26909 non-null  int32 
 6   movein_month  26909 non-null  int32 
 7   movein_year   26909 non-null  int32 
dtypes: int32(6), object(2)
memory usage: 1.0+ MB


# Treating Missing Values

In [30]:
df.isnull().sum()

status             0
lost_reason     2246
budget             0
lease              0
stay_type          0
movein_day         0
movein_month       0
movein_year        0
dtype: int64

In [31]:
df[df.lost_reason.isnull()]

,status,lost_reason,budget,lease,stay_type,movein_day,movein_month,movein_year
24663,WON,NaN,121,40,1,9,9,2022
24664,WON,NaN,181,40,1,29,9,2022
24665,WON,NaN,121,40,1,31,8,2022
24666,WON,NaN,121,50,1,7,10,2022
24667,WON,NaN,121,50,1,22,9,2022
...,...,...,...,...,...,...,...,...
26904,WON,NaN,139,42,0,10,1,2022
26905,WON,NaN,179,51,0,9,10,2022
26906,WON,NaN,180,44,0,16,9,2022
26907,WON,NaN,121,50,1,9,5,2022


In [32]:
df.lost_reason= df.lost_reason.fillna('no lost reason')

In [33]:
df.isnull().sum()

status          0
lost_reason     0
budget          0
lease           0
stay_type       0
movein_day      0
movein_month    0
movein_year     0
dtype: int64

# Mapping Status Column

In [34]:
df.status= df['status'].map({'LOST':0,'WON':1})
df.status=df.status.astype(int)
df.head()

,status,lost_reason,budget,lease,stay_type,movein_day,movein_month,movein_year
0,0,Not responding,121,40,1,31,8,2022
1,0,Wants private accommodation,181,51,0,9,8,2022
2,0,Not responding,121,40,1,15,9,2022
3,0,Wants private accommodation,121,40,1,9,2,2022
4,0,Not responding,181,50,1,9,4,2022


# OneHotEncoding

In [35]:
ohe= OneHotEncoder(drop= 'first')

In [36]:
z=ohe.fit_transform(df[['lost_reason']])

In [37]:
z=pd.DataFrame(z.toarray())
z.columns= [ 'Booked with competitor',
       'Booked with manager', 'Cross sell', "Didn't respond in time",
       'Distance issue', 'Guarantor issue', 'Inadequate details',
       'Junk lead', 'Junk lead/ Just Enquiring', 'Just Enquiring',
       'Lead issue', 'Looking for Scholarship', 'Looking for admission',
       'Looking for loan', 'Low availability', 'Low budget', 'No supply',
       'Not a student', 'Not going to university', 'Not interested',
       'Not responding', 'Not serving in that region', 'Repeat lead',
       'Semester stay', 'Short stay', 'Supply issue', 'Underage student',
       'Visa/admission denied', 'Wants private accommodation',
       'no lost reason']

In [38]:
df1= pd.concat([df,z],axis=1)
df1

,status,lost_reason,budget,lease,stay_type,movein_day,movein_month,movein_year,Booked with competitor,Booked with manager,...,Not responding,Not serving in that region,Repeat lead,Semester stay,Short stay,Supply issue,Underage student,Visa/admission denied,Wants private accommodation,no lost reason
0,0,Not responding,121,40,1,31,8,2022,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,Wants private accommodation,181,51,0,9,8,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,Not responding,121,40,1,15,9,2022,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Wants private accommodation,121,40,1,9,2,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,Not responding,181,50,1,9,4,2022,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26904,1,no lost reason,139,42,0,10,1,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26905,1,no lost reason,179,51,0,9,10,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26906,1,no lost reason,180,44,0,16,9,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26907,1,no lost reason,121,50,1,9,5,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [39]:
df1=df1.drop('lost_reason',axis=1)
df1.head()

,status,budget,lease,stay_type,movein_day,movein_month,movein_year,Booked with competitor,Booked with manager,Cross sell,...,Not responding,Not serving in that region,Repeat lead,Semester stay,Short stay,Supply issue,Underage student,Visa/admission denied,Wants private accommodation,no lost reason
0,0,121,40,1,31,8,2022,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,181,51,0,9,8,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,121,40,1,15,9,2022,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,121,40,1,9,2,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,181,50,1,9,4,2022,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Checking For Duplicates

In [40]:
df1.dtypes

status                           int32
budget                           int32
lease                            int32
stay_type                        int32
movein_day                       int32
movein_month                     int32
movein_year                      int32
Booked with competitor         float64
Booked with manager            float64
Cross sell                     float64
Didn't respond in time         float64
Distance issue                 float64
Guarantor issue                float64
Inadequate details             float64
Junk lead                      float64
Junk lead/ Just Enquiring      float64
Just Enquiring                 float64
Lead issue                     float64
Looking for Scholarship        float64
Looking for admission          float64
Looking for loan               float64
Low availability               float64
Low budget                     float64
No supply                      float64
Not a student                  float64
Not going to university  

In [41]:
df1.duplicated().sum()

15173

In [42]:
df1= df1.drop_duplicates(keep='first')
df1

,status,budget,lease,stay_type,movein_day,movein_month,movein_year,Booked with competitor,Booked with manager,Cross sell,...,Not responding,Not serving in that region,Repeat lead,Semester stay,Short stay,Supply issue,Underage student,Visa/admission denied,Wants private accommodation,no lost reason
0,0,121,40,1,31,8,2022,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,181,51,0,9,8,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,121,40,1,15,9,2022,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,121,40,1,9,2,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,181,50,1,9,4,2022,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26888,1,150,50,1,9,3,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26890,1,129,51,0,9,5,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26900,1,100,40,1,9,1,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
26903,1,135,46,0,16,9,2022,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:
df1.reset_index(drop=True, inplace= True)

In [44]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11736 entries, 0 to 11735
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   status                       11736 non-null  int32  
 1   budget                       11736 non-null  int32  
 2   lease                        11736 non-null  int32  
 3   stay_type                    11736 non-null  int32  
 4   movein_day                   11736 non-null  int32  
 5   movein_month                 11736 non-null  int32  
 6   movein_year                  11736 non-null  int32  
 7   Booked with competitor       11736 non-null  float64
 8   Booked with manager          11736 non-null  float64
 9   Cross sell                   11736 non-null  float64
 10  Didn't respond in time       11736 non-null  float64
 11  Distance issue               11736 non-null  float64
 12  Guarantor issue              11736 non-null  float64
 13  Inadequate detai

# Data is Imbalance

In [45]:
x= df1.drop('status',axis=1)
y= df1.status

In [46]:
os=SMOTETomek()
x_ne,y_ne=os.fit_resample(x,y)

In [47]:
y_ne.value_counts()

0    10729
1    10729
Name: status, dtype: int64

In [48]:
scale= MinMaxScaler()
model= scale.fit(x_ne)
x_sc=model.transform(x_ne)

# Creating Lead Score Column

In [49]:
lr= LogisticRegression()

In [50]:
xtrain,xtest,ytrain,ytest= train_test_split(x_sc,y_ne,test_size= 0.2,random_state=2)

In [51]:
lr.fit(xtrain,ytrain)

LogisticRegression()

In [52]:
ytrainpred= lr.predict(xtrain)

In [53]:
ypred=lr.predict(xtest)

In [54]:
print(classification_report(ytest,ypred))
print(confusion_matrix(ytest,ypred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2083
           1       1.00      1.00      1.00      2209

    accuracy                           1.00      4292
   macro avg       1.00      1.00      1.00      4292
weighted avg       1.00      1.00      1.00      4292

[[2083    0]
 [   0 2209]]


In [55]:
print(classification_report(ytrain,ytrainpred))
print(confusion_matrix(ytrain,ytrainpred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8646
           1       1.00      1.00      1.00      8520

    accuracy                           1.00     17166
   macro avg       1.00      1.00      1.00     17166
weighted avg       1.00      1.00      1.00     17166

[[8646    0]
 [   0 8520]]


In [56]:
np.mean(cross_val_score(lr,xtrain,ytrain,cv=10))

1.0

In [57]:
np.mean(cross_val_score(lr,xtest,ytest,cv=10))

1.0

In [58]:
lr.fit(x_sc,y_ne)

LogisticRegression()

In [59]:
print(lr.coef_)
print(lr.intercept_)

[[-7.51367910e-02  6.15470074e-03 -5.75294781e-01 -1.21827128e-01
   9.58347280e-02 -4.08102109e-02 -3.52385291e-01 -4.89744120e-01
  -3.11257744e-03 -4.56855082e-02 -2.05389003e-01 -1.48571560e-02
  -4.56621555e-01 -4.10534073e-01 -2.14312103e-01 -6.96192589e-01
  -1.74028908e-01 -1.50353371e-02 -4.65685493e-02 -4.82015530e-03
  -1.21720983e+00 -1.11255150e+00 -2.25260750e-01 -6.20759338e-01
  -2.98976110e-01 -1.20351227e+00 -1.22553864e+00 -1.24849140e-01
  -2.82306053e-01 -4.24651413e-01 -7.50372791e-01 -4.32655246e-01
  -7.80285296e-03 -2.93043922e-02 -8.64023294e-01  1.26141989e+01]]
[-5.69708076]


In [60]:
X=np.array(x_sc)

In [61]:
X=np.insert(X,0,1,axis=1)
X

array([[1.        , 0.04108659, 0.00898674, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.0614601 , 0.0114581 , ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.04108659, 0.00898674, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.0114581 , ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.0295416 , 0.00539205, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.02818336, 0.00943608, ..., 0.        , 0.        ,
        1.        ]])

In [62]:
weights=[lr.intercept_[0],lr.coef_[0][0],lr.coef_[0][1],lr.coef_[0][2],lr.coef_[0][3],lr.coef_[0][4],lr.coef_[0][5],lr.coef_[0][6],lr.coef_[0][7],lr.coef_[0][8],lr.coef_[0][9],lr.coef_[0][10],lr.coef_[0][11],lr.coef_[0][12],lr.coef_[0][13],lr.coef_[0][14],lr.coef_[0][15],lr.coef_[0][16],lr.coef_[0][17],lr.coef_[0][18],lr.coef_[0][19],lr.coef_[0][20],lr.coef_[0][21],lr.coef_[0][22],lr.coef_[0][23],lr.coef_[0][24],lr.coef_[0][25],lr.coef_[0][26],lr.coef_[0][27],lr.coef_[0][28],lr.coef_[0][29],lr.coef_[0][30],lr.coef_[0][31],lr.coef_[0][32],lr.coef_[0][33],lr.coef_[0][34],lr.coef_[0][35]]

In [63]:
weights= np.array(weights)
weights

array([-5.69708076e+00, -7.51367910e-02,  6.15470074e-03, -5.75294781e-01,
       -1.21827128e-01,  9.58347280e-02, -4.08102109e-02, -3.52385291e-01,
       -4.89744120e-01, -3.11257744e-03, -4.56855082e-02, -2.05389003e-01,
       -1.48571560e-02, -4.56621555e-01, -4.10534073e-01, -2.14312103e-01,
       -6.96192589e-01, -1.74028908e-01, -1.50353371e-02, -4.65685493e-02,
       -4.82015530e-03, -1.21720983e+00, -1.11255150e+00, -2.25260750e-01,
       -6.20759338e-01, -2.98976110e-01, -1.20351227e+00, -1.22553864e+00,
       -1.24849140e-01, -2.82306053e-01, -4.24651413e-01, -7.50372791e-01,
       -4.32655246e-01, -7.80285296e-03, -2.93043922e-02, -8.64023294e-01,
        1.26141989e+01])

In [64]:
def sigmoid(z):
    
    return 1/(1 + np.exp(-z))

z=np.dot(X,weights)
y_hat= sigmoid(z)


In [65]:
y_hat= y_hat * 100
y_hat

array([6.67245279e-02, 1.39328279e-01, 7.18232552e-02, ...,
       9.98964250e+01, 9.98979224e+01, 9.98999848e+01])

In [66]:
#for i in y_hat:
    #print(i)

In [67]:
temp3= pd.DataFrame(y_hat)

In [68]:
temp3.columns= ['lead score']
temp3

,lead score
0,0.066725
1,0.139328
2,0.071823
3,0.099368
4,0.070352
...,...
21453,99.865690
21454,99.861606
21455,99.896425
21456,99.897922


In [69]:
df= pd.concat([x_ne,temp3],axis=1)
df.head()

,budget,lease,stay_type,movein_day,movein_month,movein_year,Booked with competitor,Booked with manager,Cross sell,Didn't respond in time,...,Not serving in that region,Repeat lead,Semester stay,Short stay,Supply issue,Underage student,Visa/admission denied,Wants private accommodation,no lost reason,lead score
0,121,40,1,31,8,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066725
1,181,51,0,9,8,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.139328
2,121,40,1,15,9,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071823
3,121,40,1,9,2,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.099368
4,181,50,1,9,4,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070352


# Creating Model For Prediction of Lead Score

In [70]:
X= df.drop('lead score',axis=1)
Y= df['lead score']

In [71]:
model= scale.fit(X)
X_sc= model.transform(X)

# 1. Linear Regression

In [72]:
LR= LinearRegression()

In [73]:
xtrain,xtest,ytrain,ytest= train_test_split(X_sc,Y,test_size= 0.2,random_state= 2)

In [74]:
model=LR.fit(xtrain,ytrain)

In [75]:
ytrain_pred= model.predict(xtrain)
ytest_pred= model.predict(xtest)

In [76]:
print('R2 Score of Train Data: ',r2_score(ytrain,ytrain_pred))

R2 Score of Train Data:  0.9999999820562446


In [77]:
print('R2 Score of Test Data: ',r2_score(ytest,ytest_pred))

R2 Score of Test Data:  0.9999999828078898


In [78]:
n=11736
p=36
R2=0.99
Adj_r2 = 1-(((1-R2)*(n-1))/(n-p-1))
print('Adjusted R2 Score is: ',Adj_r2)

Adjusted R2 Score is:  0.9899692281391572


In [79]:
print('Cross Validation Score: ',np.mean(cross_val_score(LR,xtest,ytest,cv=10)))

Cross Validation Score:  0.9999999825106635


In [80]:
print('Root Mean Squared Error: ',np.sqrt(mean_squared_error(ytest,ytest_pred)))


Root Mean Squared Error:  0.006537666246726002


# 2.Random Forest Regressor

In [81]:
rf= RandomForestRegressor()

In [82]:
rf.fit(xtrain,ytrain)

RandomForestRegressor()

In [83]:
ytrain_pred = rf.predict(xtrain)
ytets_pred= rf.predict(xtest)
print('R2 Score of Train Data: ',r2_score(ytrain,ytrain_pred))
print('R2 Score of Test Data: ',r2_score(ytest,ytest_pred))

R2 Score of Train Data:  0.9999999999415996
R2 Score of Test Data:  0.9999999828078898


In [84]:
n=11736
p=36
R2=0.99
Adj_r2 = 1-(((1-R2)*(n-1))/(n-p-1))
print('Adjusted R2 Score is: ',Adj_r2)

print('Cross Validation Score: ',np.mean(cross_val_score(LR,xtest,ytest,cv=10)))

print('Root Mean Squared Error: ',np.sqrt(mean_squared_error(ytest,ytest_pred)))


Adjusted R2 Score is:  0.9899692281391572
Cross Validation Score:  0.9999999825106635
Root Mean Squared Error:  0.006537666246726002


# 3.Gradient Boosting Regressor

In [85]:
gb= GradientBoostingRegressor()

In [86]:
gb.fit(xtrain,ytrain)

GradientBoostingRegressor()

In [87]:
ytrain_pred = rf.predict(xtrain)
ytets_pred= rf.predict(xtest)
print('R2 Score of Train Data: ',r2_score(ytrain,ytrain_pred))
print('R2 Score of Test Data: ',r2_score(ytest,ytest_pred))

R2 Score of Train Data:  0.9999999999415996
R2 Score of Test Data:  0.9999999828078898


In [88]:
n=11736
p=36
R2=0.99
Adj_r2 = 1-(((1-R2)*(n-1))/(n-p-1))
print('Adjusted R2 Score is: ',Adj_r2)

print('Cross Validation Score: ',np.mean(cross_val_score(LR,xtest,ytest,cv=10)))

print('Root Mean Squared Error: ',np.sqrt(mean_squared_error(ytest,ytest_pred)))


Adjusted R2 Score is:  0.9899692281391572
Cross Validation Score:  0.9999999825106635
Root Mean Squared Error:  0.006537666246726002


# 4.XGBoost

In [89]:
xgb= XGBRegressor()

In [90]:
xgb.fit(xtrain,ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [91]:
ytrain_pred = rf.predict(xtrain)
ytets_pred= rf.predict(xtest)
print('R2 Score of Train Data: ',r2_score(ytrain,ytrain_pred))
print('R2 Score of Test Data: ',r2_score(ytest,ytest_pred))

R2 Score of Train Data:  0.9999999999415996
R2 Score of Test Data:  0.9999999828078898


In [92]:
n=11736
p=36
R2=0.99
Adj_r2 = 1-(((1-R2)*(n-1))/(n-p-1))
print('Adjusted R2 Score is: ',Adj_r2)

print('Cross Validation Score: ',np.mean(cross_val_score(LR,xtest,ytest,cv=10)))

print('Root Mean Squared Error: ',np.sqrt(mean_squared_error(ytest,ytest_pred)))


Adjusted R2 Score is:  0.9899692281391572
Cross Validation Score:  0.9999999825106635
Root Mean Squared Error:  0.006537666246726002
